In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

In [3]:
from datasets import Dataset
import pandas as pd

df=pd.read_csv('/content/Psych8k.csv')
df.drop(['Unnamed: 0','instruction'], axis=1, inplace=True)
df.head()
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['output', 'input'],
    num_rows: 8187
})

In [4]:
# Add padding token to tokenizer
tokenizer.pad_token = tokenizer.eos_token

In [5]:
def preprocess(examples):
    # Combine question and answer as a single input for conversational modeling
    inputs = [q + tokenizer.eos_token + a + tokenizer.eos_token for q, a in zip(examples['input'], examples['output'])]
    tokenized_inputs = tokenizer(inputs, truncation=True, padding=True, max_length=128)

    # Labels are the same as input_ids for causal language models
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].copy()

    return tokenized_inputs



In [6]:
data=dataset.map(preprocess, batched=True)

Map:   0%|          | 0/8187 [00:00<?, ? examples/s]

In [7]:
data

Dataset({
    features: ['output', 'input', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8187
})

In [ ]:
!pip install trl

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
training_args = TrainingArguments(
     output_dir="./results",         # Directory to save model checkpoints
    eval_strategy="no",    # Disable evaluation
    do_eval=False,  #Disable evaluation
    per_device_train_batch_size=4,  # Adjust according to your GPU memory
    per_device_eval_batch_size=4,   # Adjust according to your GPU memory
    logging_dir="./logs",           # Directory to save logs
    logging_steps=500,
    save_steps=1000,
    num_train_epochs=6,             # Number of training epochs
    report_to="none",               # Disable logging to W&B or similar platforms
)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data,
    tokenizer=tokenizer,
)

<ipython-input-9-9794b3c0456d>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
trainer.train()

Step,Training Loss
500,2.638600
1000,2.348300
1500,2.270300
2000,2.229800
2500,2.087200
3000,2.068800
3500,2.064600
4000,2.062200
4500,1.956700
5000,1.936500


TrainOutput(global_step=12282, training_loss=1.9575749359851433, metrics={'train_runtime': 2878.1438, 'train_samples_per_second': 17.067, 'train_steps_per_second': 4.267, 'total_flos': 3208796798976000.0, 'train_loss': 1.9575749359851433, 'epoch': 6.0})

In [17]:
model.save_pretrained('finetuned_dialog_gpt_small')

In [19]:

tokenizer.save_pretrained('finetuned_dialog_gpt_small_tokenizer')

('finetuned_dialog_gpt_small_tokenizer/tokenizer_config.json',
 'finetuned_dialog_gpt_small_tokenizer/special_tokens_map.json',
 'finetuned_dialog_gpt_small_tokenizer/vocab.json',
 'finetuned_dialog_gpt_small_tokenizer/merges.txt',
 'finetuned_dialog_gpt_small_tokenizer/added_tokens.json',
 'finetuned_dialog_gpt_small_tokenizer/tokenizer.json')

# New Section

In [11]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [27]:
from huggingface_hub import login
login()

In [28]:
model.push_to_hub('dialog_gpt_small_psycho8k')
trainer.push_to_hub('dialog_gpt_small_psycho8k')

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/qazws345/results/commit/1f93c1bb342a993e0cfcdf7ce417ae368a49be0e', commit_message='dialog_gpt_small_psycho8k', commit_description='', oid='1f93c1bb342a993e0cfcdf7ce417ae368a49be0e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/qazws345/results', endpoint='https://huggingface.co', repo_type='model', repo_id='qazws345/results'), pr_revision=None, pr_num=None)

In [30]:

from huggingface_hub import Repository

repo = Repository(
    local_dir="./",
    repo_type="model",
    repo_id="qazws345/results"
)

repo.rename("Dialog_GPT_Small_Psycho8k")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)


TypeError: Repository.__init__() got an unexpected keyword argument 'repo_id'

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("qazws345/DialogGPT_Psycho8k")
model = AutoModelForCausalLM.from_pretrained("qazws345/DialogGPT_Psycho8k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/903 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [38]:
model.num_parameters()


124439808

In [4]:
import torch
def chat(inp=None):
    # Check for available device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)  # Move model to appropriate device

    for step in range(5):  # Modify or remove this loop based on your needs
        if inp is None:
            user_input = input(">> User: ")  # Get input from user if not provided
        else:
            user_input = inp[step]  # Use predefined input if available

        # Ensure user input is valid
        if not user_input.strip():
            print("No input received. Please try again.")
            continue

        # Encode user input and add EOS token for the model
        inputs = tokenizer(user_input + tokenizer.eos_token, return_tensors='pt').to(device)

        # Generate the response using the model with attention_mask
        chat_history_ids = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],  # Pass the attention mask
            max_length=1500,
            pad_token_id=tokenizer.eos_token_id,  # Ensure EOS token is used for padding
            do_sample=True,
            temperature=0.6
            top_p=0.92,
            top_k=50
        )

        # Decode the bot's response
        bot_response = tokenizer.decode(chat_history_ids[0], skip_special_tokens=True)

        # Remove the user input part from the bot's response
        bot_response = bot_response[len(user_input):].strip()

        if bot_response:
            print(f"DialoGPT: {bot_response}")
        else:
            print("DialoGPT: Sorry, I couldn't understand. Can you rephrase?")


In [ ]:
chat()

>> User: Lately I've been feeling really overwhelmed with everything. I've been hearing strange noises like whispers, but I'm not sure if it's just my mind playing tricks on me. Sometimes, it's really hard for me to concentrate. I've been just drifting away in social situations
DialoGPT: It sounds like you're experiencing some unusual sensations like hearing strange noises. It's important to acknowledge these experiences and work on reducing the intensity of these disturbances. Can you think of any specific situations or triggers that might be contributing to these feelings?
